In [1]:
%qtconsole

In [1]:
import numpy as np
import tensorflow as tf
import spektral

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from spektral.layers import GCNConv, GeneralConv
from spektral.layers import GlobalSumPool
from spektral.data import DisjointLoader, Dataset
from spektral.datasets import TUDataset
from spektral.datasets.graphsage import Reddit
from spektral.layers import GraphSageConv
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense
from spektral.data import BatchLoader
from tensorflow.keras.layers import Layer, Dense, Activation, Input, Lambda, Concatenate, Dropout, ReLU, Reshape,Flatten

from tensorflow.keras.optimizers import SGD, Adam

2024-06-19 09:44:24.393566: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-19 09:44:25.211699: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
#Model creation
#Idea from joewilaj/nbaGNNs

class To_Sparse(Layer):

    def __init__(self,):
        super(To_Sparse,self).__init__()

    def call(self,inputs):

        sparse_t = tf.sparse.from_dense(inputs[0], name = 'adj_mat_sp')

        return sparse_t



class node_pair_xt(Layer):

    def __init__(self):
        super(node_pair_xt,self).__init__()

    def call(self,inputs):
        
        extracted = tf.gather(inputs[1],inputs[0], axis=1)

        return extracted


def LinkPredictor(nodes_n,features_n,ch_gnn1,ch_gnn2,drop_rate):

    #Inputs
    raw_adj = Input(shape=(nodes_n,nodes_n),name='adj_mtx')
    adjmtx = To_Sparse()(raw_adj)
    nodefeat= Input(shape=(nodes_n, features_n), name='feature_mtx')
    index_train = Input(shape=(2,),dtype = tf.int64, name='index_pair_nodes')
    
    #Layers
    conv1 = spektral.layers.GeneralConv(channels= ch_gnn1, name='gcn1')([nodefeat,adjmtx])
    conv2 = spektral.layers.GeneralConv(channels= ch_gnn2, name='gcn2')([conv1,adjmtx])
    extracted = node_pair_xt()([index_train,conv2])
    rshp = Reshape((int(np.floor(extracted.shape[2]*extracted.shape[3])),))(extracted)
    
    dense = Dense(64)(rshp)
    dense = Dropout(drop_rate)(dense)
    dense = Dense(12)(dense)
    dense = Dropout(drop_rate)(dense)
    output_layer = Dense(2, 'softmax')(dense)
    
    #Model
    model = Model(inputs= [raw_adj, nodefeat, index_train ], outputs = output_layer)
       
    return model
   

In [158]:
def EdgePredictor(nodes_n,features_n,ch_gnn1,ch_gnn2,drop_rate):

    #Inputs
    raw_adj = Input(shape=(nodes_n,nodes_n),name='adj_mtx')
    adjmtx = To_Sparse()(raw_adj)
    nodefeat= Input(shape=(nodes_n, features_n), name='feature_mtx')
    index_train = Input(shape=(2,),dtype = tf.int64, name='index_pair_nodes')
    index_filter = Input(shape=(nodes_n,ch_gnn2,),dtype = tf.int64, name='index_filter')
    
    #Layers
    conv1 = spektral.layers.GeneralConv(channels= ch_gnn1, name='gcn1')([nodefeat,adjmtx])
    conv2 = spektral.layers.GeneralConv(channels= ch_gnn2, name='gcn2')([conv1,adjmtx])
    extracted = node_pair_xt()([index_train,conv2])
    #rshp = Reshape((int(np.floor(extracted.shape[2]*extracted.shape[3])),))(extracted)
    rshp = np.take(extracted, index_filter)
    
    dense = Dense(64)(rshp)
    dense = Dropout(drop_rate)(dense)
    dense = Dense(12)(dense)
    dense = Dropout(drop_rate)(dense)
    output_layer = Dense(2, 'softmax')(dense)
    
    #Model
    model = Model(inputs= [raw_adj, nodefeat, index_train], outputs = output_layer)
       
    return model

In [159]:



link_pred = EdgePredictor(nodes_n=20, features_n=12, ch_gnn1=64, ch_gnn2=128, drop_rate=0.3)
link_pred.summary()

TypeError: You are passing KerasTensor(type_spec=TensorSpec(shape=(None, None, 2, 128), dtype=tf.float32, name=None), name='node_pair_xt_117/GatherV2:0', description="created by layer 'node_pair_xt_117'"), an intermediate Keras symbolic input/output, to a TF API that does not allow registering custom dispatchers, such as `tf.cond`, `tf.function`, gradient tapes, or `tf.map_fn`. Keras Functional model construction only supports TF API calls that *do* support dispatching, such as `tf.math.add` or `tf.reshape`. Other APIs cannot be called directly on symbolic Kerasinputs/outputs. You can work around this limitation by putting the operation in a custom Keras layer `call` and calling that layer on this symbolic input/output.

In [149]:
link_pred = LinkPredictor(nodes_n=20, features_n=12, ch_gnn1=64, ch_gnn2=128, drop_rate=0.3)
link_pred.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 adj_mtx (InputLayer)        [(None, 20, 20)]             0         []                            
                                                                                                  
 feature_mtx (InputLayer)    [(None, 20, 12)]             0         []                            
                                                                                                  
 to__sparse_4 (To_Sparse)    (20, 20)                     0         ['adj_mtx[0][0]']             
                                                                                                  
 gcn1 (GeneralConv)          (None, 20, 64)               2368      ['feature_mtx[0][0]',         
                                                                     'to__sparse_4[0][0]']  

In [4]:
opt = Adam(learning_rate=0.001)

In [6]:
def LPred_training_set(input_matrix, feat_vec, batch_size):
    
    index_train = np.zeros((batch_size,2),dtype = int)
    y_train = np.zeros((batch_size,2),dtype = float)
    adjmtx = np.zeros((batch_size, InputMatrix.shape[0],InputMatrix.shape[1]),dtype = int)
    
    #construct matrix
    
    for index in range(batch_size):
        #mask 1 edge
        #construct y (half time right/ half time false)
        
        
    
    return index_train,adjmtx,nodefeat,y_train
    

IndentationError: expected an indented block (<ipython-input-6-62145db75698>, line 15)

In [16]:
#End2end test on random matrices

Ad_input = np.random.rand(10,20,20)
Ad_input = Ad_input.round()

Feat_input = np.random.rand(10,20,12)

index_train = np.random.randint(1,20, size=(10, 2))

y=np.random.rand(10)
y=y.round()

y =tf.keras.utils.to_categorical(
    y, num_classes=None, dtype='float32')

In [161]:
#Test
Ad_input = np.random.randint(1,100, size=(3,4, 4))
#Ad_input = np.random.rand(3,4,4)
#Ad_input = Ad_input.round()
#print(Ad_input)

index_train = np.random.randint(1,4, size=(3, 2))

print(index_train)

extracted = node([index_train,Ad_input])
#extracted2 = np.take(extracted, [[0,1,2,3,4,5,6,7],[10,11,12,13,14,15,16,17]])
extracted.shape
del(tempindex)
del(out)
print(extracted)

for i in range(index_train.shape[0]):
    #print(i)
    tempindex = np.array(list(range(Ad_input.shape[2]*2)))+(i*Ad_input.shape[2]*2)
    #print(tempindex)
    #print(i)
    if i == 0:
        out = tempindex
    else:
        #print(out)
        #print(tempindex)
        #print(i)
        tempindex = tempindex+ ((Ad_input.shape[2]*2*Ad_input.shape[0])*i)
        out = np.vstack((out, tempindex))
      
        
    
    #print(tempindex)
    
    
    
#print(extracted)
#rshp = Reshape((int(np.floor(extracted.shape[2]*extracted.shape[3])),))(extracted)
#print(extracted2)
#print(rshp)
#print(out)
res = np.take(extracted, out)

print(res)
#
#res.shape
#print(out.shape)

a = tf.convert_to_tensor(extracted)
b = tf.convert_to_tensor(out)
# c = a[b] # does not work
#result_tf = tf.gather(a, b)
res2=tf.gather(extracted, out)

[[2 1]
 [1 1]
 [3 1]]
tf.Tensor(
[[[[72 56 67 16]
   [90  5 67 42]]

  [[90  5 67 42]
   [90  5 67 42]]

  [[15 19 88 77]
   [90  5 67 42]]]


 [[[48 38  5  7]
   [91 94  3 41]]

  [[91 94  3 41]
   [91 94  3 41]]

  [[60 69 93  8]
   [91 94  3 41]]]


 [[[31 17 26 78]
   [39 76 33 84]]

  [[39 76 33 84]
   [39 76 33 84]]

  [[41 20 25 68]
   [39 76 33 84]]]], shape=(3, 3, 2, 4), dtype=int64)
[[72 56 67 16 90  5 67 42]
 [91 94  3 41 91 94  3 41]
 [41 20 25 68 39 76 33 84]]


InvalidArgumentError: {{function_node __wrapped__GatherV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[0,3] = 3 is not in [0, 3) [Op:GatherV2] name: 

In [38]:
src = np.array([[[1,2,3],[4,5,6]],
                [[7,8,9],[10,11,12]]])

idx = np.array([[0,4],
                [7,11]])

# Wanted result
res = np.take(src, idx)
print(res)

[[ 1  5]
 [ 8 12]]


In [17]:
def LinkPredictor2(nodes_n,features_n,ch_gnn1,ch_gnn2,drop_rate):

    #Inputs
    raw_adj = Input(shape=(nodes_n,nodes_n),name='adj_mtx')
    adjmtx = To_Sparse()(raw_adj)
    nodefeat= Input(shape=(nodes_n, features_n), name='feature_mtx')
    index_train = Input(shape=(2,),dtype = tf.int64, name='index_pair_nodes')
    
    #Layers
    conv1 = spektral.layers.GeneralConv(channels= ch_gnn1, name='gcn1')([nodefeat,adjmtx])
    conv2 = spektral.layers.GeneralConv(channels= ch_gnn2, name='gcn2')([conv1,adjmtx])
    extracted = node_pair_xt()([index_train,conv2])
    rshp = Reshape((int(np.floor(extracted.shape[2]*extracted.shape[3])),))(extracted)
    
    dense = Dense(64)(rshp)
    dense = Dropout(drop_rate)(dense)
    dense = Dense(12)(dense)
    dense = Dropout(drop_rate)(dense)
    output_layer = Dense(2, 'softmax')(dense)
    
    #Model
    model = Model(inputs= [raw_adj, nodefeat, index_train ], outputs = output_layer)
       
    return model
   

In [6]:
#print(Ad_input[1])

no_links =np.where(Ad_input[1] == 0)
links =np.where(Ad_input[1] == 1)

for item in range(len(no_links[0])):
    if(item == 0):
        index_no_link = [no_links[0][item], no_links[1][item]]
    else:
        #print(index[0][item], index[1][item])
        index_no_link = np.vstack([index_no_link,  [no_links[0][item], no_links[1][item]] ])


for item in range(len(links[0])):
    if(item == 0):
        index_link = [links[0][item], links[1][item]]
    else:
        #print(index[0][item], index[1][item])
        index_link = np.vstack([index_link,  [links[0][item], links[1][item]] ])


In [23]:
link_pred.compile(loss='binary_crossentropy', optimizer= opt, metrics=['accuracy'])

link_pred.fit([Ad_input,Feat_input,index_train],y, epochs = 10,batch_size = 1)

link_pred.predict([Ad_input,Feat_input,index_train])

Epoch 1/10
10/10 [==============================] - 1s 2ms/step - loss: 6.0598e-19 - accuracy: 1.0000
Epoch 2/10
10/10 [==============================] - 0s 2ms/step - loss: 2.1427e-36 - accuracy: 1.0000
Epoch 3/10
10/10 [==============================] - 0s 2ms/step - loss: 3.3561e-38 - accuracy: 1.0000
Epoch 4/10
10/10 [==============================] - 0s 3ms/step - loss: 8.0975e-25 - accuracy: 1.0000
Epoch 5/10
10/10 [==============================] - 0s 2ms/step - loss: 7.7118e-24 - accuracy: 1.0000
Epoch 6/10
10/10 [==============================] - 0s 2ms/step - loss: 1.8441e-37 - accuracy: 1.0000
Epoch 7/10
10/10 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/10
10/10 [==============================] - 0s 2ms/step - loss: 1.1003e-31 - accuracy: 1.0000
Epoch 9/10
10/10 [==============================] - 0s 2ms/step - loss: 5.5276e-26 - accuracy: 1.0000
Epoch 10/10
10/10 [==============================] - 0s 2ms/step - loss: 3.9713e-2

InvalidArgumentError: Graph execution error:

Detected at node 'model/reshape/Reshape' defined at (most recent call last):
    File "/home/nacho/anaconda3/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/nacho/anaconda3/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/nacho/anaconda3/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/nacho/anaconda3/lib/python3.8/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/home/nacho/anaconda3/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/nacho/anaconda3/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/home/nacho/anaconda3/lib/python3.8/asyncio/base_events.py", li

In [ ]:
class DataGeneratorLinkPredictor(keras.utils.Sequence):
    
    def __init__(self, adj_mtx, edg_lst, zer_lst, nod_emb, batch_size=32, shuffle=True):
        self.batch_size = batch_size
        self.adj_mtx = adj_mtx
        self.edg_lst = edj_lst
        self.nod_emb = nod_emb
        self.shuffle = shuffle
        self.edg_list = edg_lst
        self.zer_list = zer_lst
        self.zeros  = zer_lst.shape[0]
        self.ones  = edg_lst.shape[0]
        self.on_epoch_end()
        

    def __len__(self):
        if(self.ones >= self.zeros):
            items_per_batch = int(np.floor(self.zeros / self.batch_size))
        else:
            items_per_batch = int(np.floor(self.ones / self.batch_size))

        return items_per_batch
            

    def __getitem__(self, index):
        
        index0 = self.indexes_0[index*self.batch_size:(index+1)*self.batch_size,]
        index1 = self.indexes_1[index*self.batch_size:(index+1)*self.batch_size,]

        # Generate data
        x_adj, y = self.__data_generation(index1,index0)

        return x_adj, y

    def on_epoch_end(self):
        ione = np.arange(self.ones)
        izero = np.arange(self.zeros)
        
        
        if self.shuffle == True:
            np.random.shuffle(ione)
            np.random.shuffle(izero)
        
        self.indexes_0 = self.zer_list[izero,]
        self.indexes_1 = self.edg_list[ione,]

    def __data_generation(self, index1, index0):

        for i in range(index0.shape[0]):
            # Store sample
            X = self.adj_mtx
            X[index0[i,]]=1
            X = np.expand_dims(X, axis=0)
            Y = 0
            if(i == 0):
                x_out = X
                y_out=0
            else:
                x_out = np.concatenate([x_out,X], axis=0)
                y_out = np.concatenat([y_out,Y])
                
        for i in range(index1.shape[0]):
            # Store sample
            X = self.adj_mtx
            X[index1[i,]]=0
            X = np.expand_dims(X, axis=0)
            Y = 1
            x_out = np.concatenate([x_out,X], axis=0)
            y_out = np.concatenat([y_out,Y])

            #shuffle
            x_out = x_out[np.random.shuffle(np.arange(x_out.shape[0])),,]


        return x_out, self.nod_emb, keras.utils.to_categorical(y_out, num_classes=self.n_classes)